In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization, ZeroPadding2D,\
    MaxPooling2D, add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/blood_cell_images.zip
!unzip -nq blood_cell_images.zip
!ls

In [ ]:
# Resize all images to this
IMAGE_SIZE = [224, 224]

# Training config
epochs     = 16
batch_size = 128

In [ ]:
train_path = 'blood_cell_images/TRAIN'
valid_path = 'blood_cell_images/TEST'

In [ ]:
# Useful for getting number of files
image_files       = glob(train_path + '/*/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')
# Useful for getting number of classes
folders = glob(train_path + '/*')

In [ ]:
# Look at an image for fun
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()

In [ ]:
def identity_block(input_, kernel_size, filters):
    # 03 filter sizes
    f1, f2, f3 = filters
    
    x = Conv2D(filters=f1, kernel_size=(1, 1), kernel_initializer='he_normal')(input_)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    x = Conv2D(filters=f2, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    x = Conv2D(filters=f3, kernel_size=(1, 1), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = add([x, input_])
    x = Activation(activation='relu')(x)
    
    return x

In [ ]:
def conv_block(input_, kernel_size, filters, strides=(2, 2)):
    f1, f2, f3 = filters
    
    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=strides, kernel_initializer='he_normal')(input_)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    x = Conv2D(filters=f2, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    x = Conv2D(filters=f3, kernel_size=(1, 1), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv2D(filters=f3, kernel_size=(1, 1), strides=strides, kernel_initializer='he_normal')(input_)
    shortcut = BatchNormalization()(shortcut)
    
    x = add([x, shortcut])
    x = Activation(activation='relu')(x)
    
    return x

In [ ]:
# Custom ResNet
i = Input(shape=IMAGE_SIZE + [3])
x = ZeroPadding2D(padding=(3, 3))(i)
x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='valid', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation(activation='relu')(x)

x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

x = conv_block    (input_=x, kernel_size=3, filters=[64, 64, 256], strides=(1, 1))
x = identity_block(input_=x, kernel_size=3, filters=[64, 64, 256])
x = identity_block(input_=x, kernel_size=3, filters=[64, 64, 256])

x = conv_block    (input_=x, kernel_size=3, filters=[128, 128, 512])
x = identity_block(input_=x, kernel_size=3, filters=[128, 128, 512])
x = identity_block(input_=x, kernel_size=3, filters=[128, 128, 512])
x = identity_block(input_=x, kernel_size=3, filters=[128, 128, 512])

# The head of NN
x = Flatten()(x)
prediction = Dense(len(folders), activation='softmax')(x)

# A model object
model = Model(inputs=i, outputs=prediction)

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# The model's structure
model.summary()

In [ ]:
# Create an instance of ImageDataGenerator
def preprocess_input2(x):
    x /= 127.5
    x -= 1.
    return x

train_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input2
)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input2)

# Create generators
train_generator = train_gen.flow_from_directory(
    directory=train_path,
    target_size=IMAGE_SIZE,
    shuffle=True,
    batch_size=batch_size,
    class_mode='sparse'
)
valid_generator = val_gen.flow_from_directory(
    directory=valid_path,
    target_size=IMAGE_SIZE,
    shuffle=True,
    batch_size=batch_size,
    class_mode='sparse'
)

In [ ]:
# Fit the model
checkpoint_filepath = '/tmp/checkpoint'
r = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    steps_per_epoch= len(image_files)       // batch_size,
    validation_steps=len(valid_image_files) // batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True)
    ]
)